In [1]:
from langchain_openai import ChatOpenAI

In [2]:
llm = ChatOpenAI(
    model="qwen2.5-3b-instruct-q4_k_m.gguf",
    base_url="http://localhost:8002/v1",
    api_key="EMPTY",   # 실제로 사용되지 않음
    temperature=0.2,
)

In [3]:
response = llm.invoke("LangChain이 무엇인지 한 문장으로 설명해줘")
print(response.content)

LangChain은 Chain of Reasoning with Language Models을 의미하며, 여러 언어 모델을 연결하여 문제 해결과 정보 추출을 돕는 기반 플랫폼입니다.


In [13]:
from langchain_core.prompts import PromptTemplate
#  PromptTemplate과 ChatPromptTemplate은 동일하게 동작하나, 단일문자열 or 역할기반 메시지의 차이임.

prompt = PromptTemplate(
    template="""
    당신은 AI 튜터입니다.
    아래 질문에 대해 한 문장으로 명확하게 한글로 답하세요.
    
    질문: {question}
    """,
    input_variables=["question"],
)

print(prompt.format(question="LangChain이 왜 필요한가?"))


    당신은 AI 튜터입니다.
    아래 질문에 대해 한 문장으로 명확하게 한글로 답하세요.
    
    질문: LangChain이 왜 필요한가?
    


In [14]:
llm.invoke(prompt.format(question="Ontology가 무엇인가?"))

AIMessage(content='Ontology는 정보의 구조와 의미를 정의하고 관계를 명확히 하는 체계적인 모델 또는 체계입니다.', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 68, 'total_tokens': 101, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen2.5-3b-instruct-q4_k_m.gguf', 'system_fingerprint': 'b7654-3333951d8', 'id': 'chatcmpl-tbi8iJCUnySyLWL68p0Y9R7NLmqpZUAp', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b9910-2c55-7371-abb8-087b29ff5726-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 68, 'output_tokens': 33, 'total_tokens': 101, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a professional translator."),
    ("human", "Translate the following text to Korean:\n{text}")
])

prompt.format_messages(text="Hello world")

[SystemMessage(content='You are a professional translator.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Translate the following text to Korean:\nHello world', additional_kwargs={}, response_metadata={})]

In [7]:
chain = prompt | llm
chain.invoke({"text": "Hello"})

AIMessage(content='Hello is translated to Korean as "안녕하세요" (annyeonghaseyo).', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 27, 'total_tokens': 48, 'completion_tokens_details': None, 'prompt_tokens_details': None}, 'model_provider': 'openai', 'model_name': 'qwen2.5-3b-instruct-q4_k_m.gguf', 'system_fingerprint': 'b7654-3333951d8', 'id': 'chatcmpl-35YCXEEKqwVgBlEvPOvzHZw5rhFcVcqm', 'finish_reason': 'stop', 'logprobs': None}, id='lc_run--019b990d-dc2d-7a02-98ac-c57413dea261-0', tool_calls=[], invalid_tool_calls=[], usage_metadata={'input_tokens': 27, 'output_tokens': 21, 'total_tokens': 48, 'input_token_details': {}, 'output_token_details': {}})

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다."),
    ("user", "{question}")
])

# messages = chat_prompt.invoke(
#     {"question": "LangChain은 무엇인가요?"}
# )

chain = chat_prompt | llm
response = chain.invoke({"question": "독도는 어느 나라 땅 인가요?"})

print(response.content)


독도는 대한민국의 땅입니다. 독도는 경상북도 울릉군에 속해 있으며, 대한민국의 영토입니다.


StrOutputParser

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser


chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다."),
    ("user", "{question}")
])

# outputparser 추가
output_parser = StrOutputParser()

chain = chat_prompt | llm | output_parser
result = chain.invoke({"question": "독도는 어느 나라 땅 인가요?"})

print(result)       # 출력은 문자열(content 속성 없음)

독도는 대한민국의 땅입니다. 독도는 경상북도 울릉군에 속해 있습니다.


RunnablePassthrough

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

chat_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 지리학자입니다."),
    ("user", "{question}")
])

chain = (
    {"question": RunnablePassthrough()}
    | chat_prompt
    | llm
    | StrOutputParser()
)

result = chain.invoke("독도는 어느 나라 땅 인가요?")     # {"question": "..."} 형태로 감싸지 않음 → RunnablePassthrough에서 다음 단계로 그대로 전달
print(result)

독도는 대한민국의 땅입니다. 독도는 경상북도 울릉군에 속해 있으며, 대한민국의 영토입니다.


OutputParser 활용 추가

In [ ]:
from langchain_core.output_parsers import JsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

parser = JsonOutputParser()

json_prompt = ChatPromptTemplate.from_messages([
    ("system", "당신은 정보를 JSON으로만 출력하는 AI입니다."),
    ("user", "질문에 답하세요.\n\n{format_instructions}\n\n질문: {question}")
])

chain = json_prompt | llm | parser 
result = chain.invoke(
    {"question": "독도는 어느 나라 땅 인가요? 국가와 위치, 역사 등을 포함해서 알려주세요.",
     "format_instructions": parser.get_format_instructions()}
)

print(result)

# 결과를 보면 알 수 있듯이, llm이 판단하여 JSON 형식으로 출력을 생성함.

{'location': {'country': '대한민국', 'island_name': '독도', 'coordinates': {'latitude': '36.2500', 'longitude': '126.8000'}}, 'history': {'establishment': '1416년', 'establisher': '조선 왕조', 'significance': '국土의 확장과 통일의象征'}, 'current_status': {'administered_by': '대한민국 정부', 'accessibility': '독도는 현재 일반 관광객의 접근이 제한됩니다.'}}


Pydantic OutputParser (Custom Setting)

In [ ]:
# Pydantic 모델 정의
from pydantic import BaseModel, Field

class Answer(BaseModel):
    summary: str = Field(description="한 문장 요약")
    keywords: list[str] = Field(description="핵심 키워드 목록")


# Parser 생성
from langchain_core.output_parsers import PydanticOutputParser

parser = PydanticOutputParser(pydantic_object=Answer)


# Prompt 구성
prompt = ChatPromptTemplate.from_messages([
    ("system", "출력을 반드시 지정된 형식으로 작성하세요."),
    ("user", "{format_instructions}\n\n질문: {question}")
])

chain = prompt | llm | parser

result = chain.invoke({
    "question": "LangChain의 역할을 설명해줘",
    "format_instructions": parser.get_format_instructions()
})

print(result)
print(type(result))

OutputParserException: Invalid json output: LangChain의 역할은 자연어 처리(NLP)와 인공지능(AI)을 통합하여 사용자와 AI 간의 대화형 인터페이스를 개선하는 데 중점을 둔다. 이를 통해 AI가 사용자의 요청에 대한 자연스러운 대화를 생성하고, 사용자가 제공한 정보를 처리하여 필요한 결과를 제공할 수 있도록 한다.

LangChain은 다양한 NLP 모듈과 기능을 제공하여 AI가 사용자의 요청을 이해하고 대답하는 데 필요한 정보를 추출하고 처리할 수 있도록 한다. 이를 통해 AI는 사용자의 요청을 이해하고, 필요한 정보를 추출하고, 이를 바탕으로 답변을 생성할 수 있다.
For troubleshooting, visit: https://docs.langchain.com/oss/python/langchain/errors/OUTPUT_PARSING_FAILURE 

SimpleJsonOutputParser

In [31]:
from langchain_core.output_parsers import SimpleJsonOutputParser

parser = SimpleJsonOutputParser()

chain = chat_prompt | llm | parser

result = chain.invoke(
    {"question": "AI의 장단점을 JSON으로 정리해줘"}
)

print(result)

{'AI': {'Strengths': [{'Name': 'Speed and Efficiency', 'Description': 'AI can process and analyze large amounts of data much faster than humans.'}, {'Name': 'Accuracy', 'Description': 'AI can perform tasks with high accuracy and consistency, reducing human error.'}, {'Name': 'Automation', 'Description': 'AI can automate repetitive and time-consuming tasks, freeing up human resources.'}, {'Name': 'Insight and Pattern Recognition', 'Description': 'AI can identify patterns and insights in data that might be difficult for humans to detect.'}], 'Weaknesses': [{'Name': 'Lack of Creativity', 'Description': 'AI lacks creativity and cannot generate new ideas or concepts.'}, {'Name': 'Limited Understanding of Context', 'Description': 'AI may struggle with tasks that require understanding of context and human emotions.'}, {'Name': 'Bias and Fairness', 'Description': 'AI systems can be biased if the data they are trained on is biased.'}, {'Name': 'Dependence on Data Quality', 'Description': 'AI sy

In [32]:
from langchain_core.output_parsers import SimpleJsonOutputParser
from langchain_core.prompts import ChatPromptTemplate

parser = SimpleJsonOutputParser()

prompt = ChatPromptTemplate.from_messages([
    ("system", "반드시 JSON 형식으로만 응답하세요."),
    ("user", """
질문에 답하세요.

JSON 형식:
{{
  "thought": "...",
  "answer": "..."
}}


질문: {question}
""")
])


### 주의! Prompt에 JSON / dict / code block을 넣을 경우 {}는 항상 {{}}로 쓴다(이스케이프) ###

chain = prompt | llm | parser

for chunk in chain.stream({"question": "독도는 어느 나라 땅 인가요?"}):
    print(chunk)

{}
{'thought': ''}
{'thought': '독'}
{'thought': '독도'}
{'thought': '독도는'}
{'thought': '독도는 대한'}
{'thought': '독도는 대한민'}
{'thought': '독도는 대한민국'}
{'thought': '독도는 대한민국의'}
{'thought': '독도는 대한민국의 땅'}
{'thought': '독도는 대한민국의 땅으로'}
{'thought': '독도는 대한민국의 땅으로,'}
{'thought': '독도는 대한민국의 땅으로, 경'}
{'thought': '독도는 대한민국의 땅으로, 경상'}
{'thought': '독도는 대한민국의 땅으로, 경상북'}
{'thought': '독도는 대한민국의 땅으로, 경상북도'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': ''}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': '독'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': '독도'}
{'thought': '독도는 대한민국의 땅으로, 경상북도 울릉군에 속해 있습니다.', 'answer': '독도는'}
{'thought': '독도는 대한

ChatMessageHistory

In [33]:
# <이전 대화를 포함한 메시지 전달>

# 라이브러리 불러오기
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI


# 프롬프트 템플릿 정의: 금융 상담 역할
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "당신은 금융 상담사입니다. 사용자에게 최선의 금융 조언을 제공합니다."),
        ("placeholder", "{messages}"),  # 대화 이력 추가
    ]
)

# 프롬프트와 모델을 연결하여 체인 생성
chain = prompt | llm

# 이전 대화를 포함한 메시지 전달
ai_msg = chain.invoke(
    {
        "messages": [
            ("human", "저축을 늘리기 위해 무엇을 할 수 있나요?"),  # 사용자의 첫 질문
            ("ai", "저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요."),  # 챗봇의 답변
            ("human", "방금 뭐라고 했나요?"),  # 사용자의 재확인 질문
        ],
    }
)
print(ai_msg.content)  # 챗봇의 응답 출력

저축을 늘리기 위해 매달 일정 금액을 자동으로 저축하는 것이 좋습니다.


In [34]:
# <`ChatMessageHistory`를 사용한 메시지 관리>

from langchain_community.chat_message_histories import ChatMessageHistory

# 대화 이력 저장을 위한 클래스 초기화
chat_history = ChatMessageHistory()

# 사용자 메시지 추가
chat_history.add_user_message("저축을 늘리기 위해 무엇을 할 수 있나요?")
chat_history.add_ai_message("저축 목표를 설정하고, 매달 자동 이체로 일정 금액을 저축하세요.")

# 새로운 질문 추가 후 다시 체인 실행
chat_history.add_user_message("방금 뭐라고 했나요?")
ai_response = chain.invoke({"messages": chat_history.messages})
print(ai_response.content)  # 챗봇은 이전 메시지를 기억하여 답변합니다.

저축을 늘리기 위해 매달 일정 금액을 자동으로 저축하는 것이 좋습니다.
